In [1]:
#!/usr/bin/env python
# coding: utf-8

# # Decision Tree 


import pandas as pd
import numpy as np
from IPython.display import Image



import os
os.chdir(r"C:\\python\\datamining\\tree_gain_ratio")
workingdir = os.path.join(os.getcwd(), '.')
os.chdir(os.path.join(workingdir, 'modules' )) 


for count in range(1,31):
    # dataset 소환
    print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
    print(count)
    data = pd.read_csv(r'C:\\python\\datamining\\DM_dataset\\DM_dataset\\classification_data\\Australian Credit Approval\\australian.csv')


    # y 기준 내림차순 정렬
    data.sort_values(by="target", ascending=True, inplace=True)
    # index 재설정
    data = data.reset_index(drop=True)




    # target data 이름 넣기 #
    target_att = 'target'

    X = data.loc[:, data.columns != target_att]
    y = data.loc[:, data.columns == target_att]



    # 기존 sklearn에 구현된 모델용 설정 # 

    ## y의 name 
    target_att = 'target'

    ## x의 name
    in_feature = list(data.columns [data.columns != target_att])

    ## data column 중 categorical variable 찾기
    cate_col = [col for col in in_feature if not np.issubdtype(X[col].dtype,     np.number)]

    ## X의 string 값을 dummey variable로 변환 
    X_dummies= pd.get_dummies(X.loc[:,in_feature], columns=cate_col)
    feature_names = X_dummies.columns

    ## y의 string 값을 numeric으로 변환 
    y, class_names = pd.factorize(data[target_att])

    from sklearn.model_selection import train_test_split  
    ## train, test = 8:2, random split
    X_train, X_test, y_train, y_test = train_test_split(X_dummies, y, test_size=0.2)




    import os
    os.chdir(r"C:\\python\\datamining\\tree_gain_ratio")
    workingdir = os.path.join(os.getcwd(), '.')
    os.chdir(os.path.join(workingdir, 'modules' )) 


    # entropy_gain을 구하기 위한 .py파일 # 총 4개(파일 4개가 연동되어있으니 수정은 삼가해주길 바람)
    from usertree import userTree as utr
    import utils




    # DT termination criteria #
    ## MAX_DEPTH: 기존 모델의 max_depth와 같음
    ## min_samples_leaf: 기존모델의 min_samples_leaf와 같음
    # -> MAX_DEPTH는 변경불가 
    MAX_DEPTH = 1000

    sample_ratio = 0.02
    min_sample_leaf = round(sample_ratio * len(X_train))

    # data를 data frame형태로 train, test로 나눔
    train_idx, test_idx = X_train.index, X_test.index
    train, test = data.loc[train_idx,:], data.loc[test_idx,:]




    # 이 부분 꼭 해주어야함 #
    train = train.sort_index()


    #  실제 데이터의 class별 각 갯수 #
    ## original y의 class 수 ##

    # data y의 이름을 넣으면 됌 #
    uni_class = np.unique(data['target'])

    class_number = {}
    for i in uni_class:    
        class_number[i] = len(data[data['target'] == i])





    # 실제 train 데이터의 class별 각 갯수 #

    # data의 y의 이름을 넣으면 됌 #
    tra_uni_class = np.unique(train['target'])
    tra_class_number = {}
    for i in tra_uni_class:    
        tra_class_number[i] = len(train[train['target'] == i])





    # 모델 생성 #
    ## min_samples_leaf: 노드의 최소 데이터 수
    ## MAX_DEPTH: 나무의 가지 수
    ## params: 분기기준 -> entropy, gini, entropy_GR 총 3개 사용가능
    ## simplify: 각 어떤 노드를 기준으로 child node로 분기 했을 때 child node의 분기 예측값이 같다면 분기하지않고 이전 노드를 유지함
    # -> simplify는 True로 설정해주세요.

    #CART_gini_ins = utr(min_sample_leaf, MAX_DEPTH, params ='entropy', simplify=True)
    #CART_gini_ins = utr(min_sample_leaf, MAX_DEPTH, params ='gini', simplify=True)
    CART_gini_ins = utr(min_sample_leaf, MAX_DEPTH, params ='entropy_GR', simplify=True)




    # 모델 피팅 # 
    ## train: dataframe형태로 된 data
    ## target_attribute_name: target data로 설정된 y값 이름
    gini_tree, gini_pprint_tree= CART_gini_ins.fit(train, target_attribute_name = "target")  





    ## CART_gini_tree: tree에서 분리된 노드들의 분기기준과 data의 index, target(=class)가 출력 -> 성능측정용
    ## CART_gini_graph_tree: tree 그래프를 생성하기 위한 값 저장 
    CART_gini_tree, CART_gini_graph_tree =                 CART_gini_ins.tree, CART_gini_ins.graph_tree   





    ## utils.get_usrt_info 코드 설명 ##
    # train: data의 train data
    # CART_gini_tree: tree에서 분리된 노드들의 분기기준과 data의 index, target(=class)가 출력 -> 성능측정용
    # target_att: data의 y이름

    print(utils.get_usrt_info(train ,CART_gini_tree, target_att = "target"))





    import graphviz
    graph_dir = 'graph'
    node, edge = CART_gini_ins.graph.tree_to_graph(CART_gini_graph_tree)
    tree_graph = graphviz.Source(node + edge+'\n}')
    tree_graph




    # PDF 파일 출력 #
    graph_dir = 'C:\\Users\SinA\\Desktop\\gainration'
    split = 'entropy_GR'
    d_set_name = 'australia_credit_approval'
    
    tree_graph.render('{}/CART_{}_{}_test_{}'.format(graph_dir, split,d_set_name,count))


    # ## 성능 측정 ##
    # > CART_gini_ins.predict에 Dataframe형태의 x,y가 동시에 존재하는 train or test data와 이전에 선언한 CART_entropy_GR_tree를 넣으면, \
    # > predict, predict probability가 각각 생성됨 

    # In[25]:


    # 성능측정 #
    ## train predict ##
    train = data.loc[train_idx,:]

    train_CART_gini_all_pred, train_CART_gini_all_pred_prob     = CART_gini_ins.predict(train, CART_gini_tree)

    ## test predict ## 
    test_CART_gini_all_pred, test_CART_gini_all_pred_prob     = CART_gini_ins.predict(test, CART_gini_tree)


    # > utils.perform_check에 train or test data의 y값, predict, probability, class의 갯수,  CLASS_DICT_를 넣으면 \
    # > train or test data의 [accuracy, recall, precision, f1, auc] 순으로 값이 생성 
    # 

    # In[26]:


    # y의 name 넣기 
    classes = np.unique(train['target'])

    # train metric
    train_CART_gini_all_met = utils.perform_check(train['target'],          train_CART_gini_all_pred,          train_CART_gini_all_pred_prob,         len(classes), CART_gini_ins.CLASS_DICT_)

    # test metric
    test_CART_gini_all_met = utils.perform_check(test['target'],          test_CART_gini_all_pred,          test_CART_gini_all_pred_prob,          len(classes), CART_gini_ins.CLASS_DICT_)


    # In[27]:


    # [accuracy, recall, precision, f1, auc] 순
    train_CART_gini_all_met


    # In[28]:


    # [accuracy, recall, precision, f1, auc] 순
    test_CART_gini_all_met


    # ### [accuracy, recall, precision, f1, auc] 순

    # In[29]:


    perform_base_str = '{} : ACCURACY :{}, RECALL :{}, PRECISION : {}, F1 : {}, AUC : {}'
    print(perform_base_str.format('\nMetric\n ',             *np.round(np.array(train_CART_gini_all_met), 3)))


    # In[30]:


    perform_base_str = '{} : ACCURACY :{}, RECALL :{}, PRECISION : {}, F1 : {}, AUC : {}'
    print(perform_base_str.format('\nMetric\n ',             *np.round(np.array(test_CART_gini_all_met), 3)))









@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
1
class prior: [0.5688405797101449, 0.4311594202898551]
leaf node별 class수: [10, 1]
leaf node별 class수: [7, 8]
leaf node별 class수: [233, 11]
leaf node별 class수: [0, 11]
leaf node별 class수: [25, 9]
leaf node별 class수: [6, 22]
leaf node별 class수: [9, 3]
leaf node별 class수: [5, 6]
leaf node별 class수: [18, 80]
leaf node별 class수: [1, 87]
   pred  depth  homogeneity    lift  coverage  number_of_variable
0     0      2        0.955  1.6789     0.442                   2
1     1      2        0.989  2.2938     0.159                   2
2     0      3        0.909  1.5980     0.020                   3
3     1      3        0.533  1.2362     0.027                   3
4     1      3        1.000  2.3193     0.020                   3
5     1      4        0.816  1.8926     0.178                   4
6     0      5        0.735  1.2921     0.062                   5

Metric
  : ACCURACY :0.889, RECALL :0.899, PRECISION : 0.853, F1 : 0.875, AUC : 0.938

Metric
  : ACCURACY :0.906,


Metric
  : ACCURACY :0.902, RECALL :0.871, PRECISION : 0.894, F1 : 0.883, AUC : 0.941

Metric
  : ACCURACY :0.848, RECALL :0.838, PRECISION : 0.873, F1 : 0.855, AUC : 0.896
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
9
class prior: [0.552536231884058, 0.447463768115942]
leaf node별 class수: [11, 0]
leaf node별 class수: [6, 10]
leaf node별 class수: [230, 9]
leaf node별 class수: [15, 4]
leaf node별 class수: [4, 22]
leaf node별 class수: [10, 2]
leaf node별 class수: [6, 11]
leaf node별 class수: [8, 3]
leaf node별 class수: [1, 17]
leaf node별 class수: [14, 169]
   pred  depth  homogeneity    lift  coverage  number_of_variable
0     0      2        0.962  1.7411     0.433                   2
1     1      2        0.923  2.0627     0.332                   2
2     0      3        1.000  1.8098     0.020                   3
3     1      3        0.625  1.3968     0.029                   3
4     1      3        0.944  2.1097     0.033                   3
5     0      4        0.789  1.4280     0.034                   4
6     1


Metric
  : ACCURACY :0.899, RECALL :0.848, PRECISION : 0.916, F1 : 0.881, AUC : 0.949

Metric
  : ACCURACY :0.855, RECALL :0.889, PRECISION : 0.812, F1 : 0.848, AUC : 0.944
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
17
class prior: [0.5489130434782609, 0.45108695652173914]
leaf node별 class수: [250, 17]
leaf node별 class수: [4, 13]
leaf node별 class수: [18, 3]
leaf node별 class수: [7, 9]
leaf node별 class수: [3, 10]
leaf node별 class수: [7, 6]
leaf node별 class수: [10, 43]
leaf node별 class수: [3, 45]
leaf node별 class수: [1, 103]
   pred  depth  homogeneity    lift  coverage  number_of_variable
0     0      1        0.936  1.7052     0.484                   1
1     1      2        0.990  2.1947     0.188                   2
2     1      3        0.938  2.0794     0.087                   3
3     1      4        0.811  1.7979     0.096                   4
4     1      5        0.765  1.6959     0.031                   5

Metric
  : ACCURACY :0.902, RECALL :0.896, PRECISION : 0.888, F1 : 0.892, AUC : 0.94

Metric
  


Metric
  : ACCURACY :0.902, RECALL :0.929, PRECISION : 0.857, F1 : 0.892, AUC : 0.933

Metric
  : ACCURACY :0.841, RECALL :0.912, PRECISION : 0.795, F1 : 0.849, AUC : 0.914
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
25
class prior: [0.5380434782608695, 0.46195652173913043]
leaf node별 class수: [234, 18]
leaf node별 class수: [0, 11]
leaf node별 class수: [7, 6]
leaf node별 class수: [4, 19]
leaf node별 class수: [30, 9]
leaf node별 class수: [4, 7]
leaf node별 class수: [0, 34]
leaf node별 class수: [9, 7]
leaf node별 class수: [8, 25]
leaf node별 class수: [0, 18]
leaf node별 class수: [1, 101]
   pred  depth  homogeneity    lift  coverage  number_of_variable
0     0      1        0.929  1.7266     0.457                   1
1     1      2        0.990  2.1431     0.185                   2
2     1      3        1.000  2.1647     0.020                   3
3     1      5        1.000  2.1647     0.062                   5

Metric
  : ACCURACY :0.897, RECALL :0.843, PRECISION : 0.927, F1 : 0.883, AUC : 0.941

Metric
  : ACCURACY :0